In [2]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import EulerDiscreteScheduler # Import samplers
from diffusers import DPMSolverMultistepScheduler # Import samplers
from diffusers.utils import load_image
from tqdm import tqdm

schedulers (samplers): https://huggingface.co/docs/diffusers/api/schedulers/overview

In [3]:
print("Torch version:",torch.__version__)
print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.2.2+cu121
Is CUDA enabled? True


https://thepythoncode.com/article/control-generated-images-with-controlnet-with-huggingface#code

# First Pipeline: ControlNet + Stable Diffusion 

In [7]:
# ControlNet model
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble", torch_dtype=torch.float16)

# Define stable diffusion pipeline with controlnet
# We use the realistic-vision-v20-2047 model for this example (finetunned on realistic images of people)
pipe = StableDiffusionControlNetPipeline.from_pretrained("stablediffusionapi/realistic-vision-v20-2047", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

# Enable efficient implementations using xformers for faster inference
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

text_encoder\model.safetensors not found
Loading pipeline components...: 100%|██████████| 6/6 [00:08<00:00,  1.37s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [9]:
# Load wireframe image
image_input = load_image("wireframes\\img5_5.png")

# Prompt
prompt = "A mid shot view of an old woman holding a black cardboard box with a barcode, parcel from the Amazon webshop in her hands, standing in front of a busy city street and people background, the weather is rainy, high photorealistic quality."

# Run the pipeline
image_output = pipe(prompt, image_input, num_inference_steps=30).images[0]

# Save the output
image_output.save("test.jpg")
print("Save test.jpg!")

100%|██████████| 30/30 [00:15<00:00,  1.93it/s]


Save test.jpg!
